# Geradores

*Geradores*, ou *iteradores*, são objetos que entregam uma sequência de valores, um por vez, quando requisitados. Os valores podem estar armazenados em memória ou serem gerados quando solicitados.

## 1. Geradores pré-definidos

A função `map`, que aplica uma função fornecida a todos os valores passados, retorna um objeto gerador. Isto é, a função será aplicada aos valores na medida do necessário, isto é, os valores serão calculados a cada vez que um novo valor for pedido.

In [ ]:
map(lambda x: 2*x - 3, [1, 4, 7, 9])

Não acredita? Então veja as temporizações abaixo:

In [ ]:
%timeit map(lambda x: x**2, range(10))

In [ ]:
%timeit map(lambda x: x**2, range(1000))

In [ ]:
%timeit map(lambda x: x**2, range(10000000))

Se queremos ver todos os valores gerados, podemos converter o gerador para uma lista. O método de converter para lista solicita ao gerador os valores um por vez até que não haja mais valores a serem gerados.

In [ ]:
list(map(lambda x: 2*x - 3, [1, 4, 7, 9]))

Outra função que retorna um gerador é a função `filter`.

In [ ]:
filter(lambda x: x ** 2 < 100, range(20))

A função `zip` também retorna um gerador:

In [ ]:
zip(range(10), range(0, 100, 10))

Vamos parar de falar sobre funções que retornam geradores, pois elas são muitas. Ao usar uma função em Python, é interessante saber se ela retorna um gerador ou não. Isso é indicado na documentação de Python, mas você também pode testar diretamente no interpretador.

## 2. Geradores criados pelo usuário

Podemos definir geradores de três formas:

1. Expressões geradoras
1. Funções geradoras
1. Objetos geradores

O terceiro caso estudaremos mais tarde, depois de falar sobre orientação a objetos. Vamos ver os dois primeiros agora (que na verdade são apenas casos especiais do terceiro).

### 2.1. Expressões geradoras

Expressões geradoras são criadas substituindo os `[]` de uma *list comprehension* por `()`.

Abaixo temos uma *list comprehension*, que retorna uma lista com todos os valores:

In [ ]:
[i ** 2 for i in range(10)]

Podemos criar uma expressão geradora com os mesmo valores usando `()`:

In [ ]:
valores = (i ** 2 for i in range(10))

Isso é agora um gerador:

In [ ]:
valores

Esse objeto gerador pode, por exemplo, ser usado num `for`:

In [ ]:
for x in valores:
    print(x)

Os parêntesis da expressão geradora podem ser **omitidos** em locais onde a sintaxe do Python já exige parêntesis, por exemplo, na passagem de parâmetros para funções ou métodos:

In [ ]:
' '.join(v.strip().upper() for v in 'a  , b  ,c, x,   y,z'.split(','))

In [ ]:
sum(x ** 2 for x in range(10000000))

In [ ]:
values = [-4, -1,-2,3]
sorted(2 * y - z for y, z in zip((abs(x) for x in values), values))

### 2.2. Funções geradoras

Funções geradoras são funções que criam um objeto gerador. Podemos conseguir uma função geradora usando `yield` ao invés de `return` para retornar um valor. Neste caso, o valor fornecido no `yield` será retornado, mas a função continuará ativa (não termina ainda). Quando for solicitado um novo valor, a função voltará a executar **continuando da instrução seguinte ao `yield`**.

Por exemplo, a função geradora abaixo gera os `n` primeiros número pares a partir de `0` (e mostra uma mensagem para ajudar a entender o que está acontecendo):

In [ ]:
def even_numbers(n):
    for current in range(0, n, 2):
        print('About to return', current)
        yield current

Quando a execução chega em `yield current` o valor atual da variável `current` é retornado como o próximo número gerado, e a execução da função é suspensa. Quando o código que usa o gerador solicitar o próximo valor, a execução retorna no comando seguinte ao `yield` (neste caso, retorna para pegar o próximo valor do `for`), como se não tivesse sido interrompida.

A função pára de gerar novos números quando termina (por chegar ao fim do código ou por executar um `return`).

Note que ao chamarmos a função, ela retorna um gerador:

In [ ]:
even_numbers(10)

In [ ]:
for x in even_numbers(5):
    print('The value is now', x)

A função pode ter o número de comandos `yield` que forem necessários. A execução é interrompida num `yield`, retorna no comando seguinte e continua até encontrar o próximo `yield`.

Por exemplo, o gerador abaixo retorna alternadamente `'😀'` e `'😩'`.

In [ ]:
def bipolar(n):
    for i in range(n):
        yield '😀'
        yield '😩'

In [ ]:
for mood in bipolar(5):
    print(mood)

Vejamos agora uma comparação de desempenho de geradores e listas. Primeiro, duas funções idênticas, uma retornando uma lista de valores e outra sendo um gerador para a mesma sequência de valores.

In [ ]:
def get_list(n):
    current = 0
    res = []
    for i in range(1, n):
        current += i * i
        res.append(current)
    return res

In [ ]:
def get_generator(n):
    current = 0
    for i in range(1, n):
        current += i * i
        yield current

Vemos que as duas geram os mesmo valores, mas de forma diferente:

In [ ]:
get_list(10)

In [ ]:
list(get_generator(10))

Para estas funções, o uso de um gerador evita a criação de uma lista muito grande, caso `n` seja grande.

Agora vamos temporizar uma operação sobre esses valores (soma) de duas formas:

In [ ]:
%timeit sum(get_list(1000000))

In [ ]:
%timeit sum(get_generator(1000000))

Vemos que neste caso o uso de geradores é ligeiramente mais eficiente.

Por fim, vejamos uma função geradora para os `n` primeiros números da sequência de Fibonacci começada em 1, 1.

In [ ]:
def fibonacci(n):
    a, b = 1, 1
    for i in range(n):
        yield a
        a, b = b, a + b

In [ ]:
list(fibonacci(10))

Como geradores podem ser infinitos, podemos definir ao invés um gerador que gera toda a sequência de Fibonacci:

In [1]:
def all_fibonacci():
    a, b = 1, 1
    while True:
        yield a
        a, b = b, a + b

Esse gerador nunca acaba, então seria um erro usá-lo diretamente num `for`:

```
# DON'T RUN THIS!
for f in all_fibonacci():
    pass
```

Mas então como podemos usar este gerador? Existem diversas formas, uma forma é usar em algum contexto onde apenas um certo número de valores é requerido:

In [2]:
for i, fib in zip(range(10), all_fibonacci()):
    print(f'The {i}-th element of the Fibonacci sequence is {fib}')

The 0-th element of the Fibonacci sequence is 1
The 1-th element of the Fibonacci sequence is 1
The 2-th element of the Fibonacci sequence is 2
The 3-th element of the Fibonacci sequence is 3
The 4-th element of the Fibonacci sequence is 5
The 5-th element of the Fibonacci sequence is 8
The 6-th element of the Fibonacci sequence is 13
The 7-th element of the Fibonacci sequence is 21
The 8-th element of the Fibonacci sequence is 34
The 9-th element of the Fibonacci sequence is 55


Neste código, como o `zip` termina assim que a sequência mais curta termina, apenas 10 valores serão requisitados para o gerador `all_fibonacci`.

## 3. Exaustão de geradores

Ao usar geradores é importante lembrar que um gerador irá fornecer valores até terminar. Após fornecido o último valor, ele não fornecerá mais valores (dizemos que ele foi exaurido).

Veja por exemplo o código abaixo.

In [ ]:
bipolar2 = bipolar(2) # Cria um objeto gerador e guarda referência
print('Phase 1')
for mood in bipolar2:
    print(mood)
print('Phase 2')
for mood in bipolar2:
    print(mood)
print('Done')

Isso acontece com todos os geradores. Por exemplo, o `map`.

In [ ]:
cubes = map(lambda x: x ** 3, range(5))
print('First time')
for q in cubes:
    print(q)
print('Second time')
for q in cubes:
    print(q)
print('Done')

Isso também acontece com o `zip`:

In [ ]:
z1 = zip([1, 2], [3, 4])
for i, j in z1:
    print(i, j)
for i, j in z1:
    print(10 * i, 10 * j)    

Portanto, sempre tome cuidado com a exaustão de geradores!

# Exercícios

1. Escreva uma função geradora que quando chamada como `sum_squares(n)` retorna um gerador dos `n` primeiros elementos da sequência definida por:

    $$x_i = \sum_{j=1}^{i} j^2.$$

    começando em $i=1$. Por exemplo, o código
    ```python
    for x in sum_squares(5):
        print(x)
    ```
    irá imprimir
    ```
    1
    5
    14
    30
    55
    ```

2. Escreva uma função geradora `primes()` que retorne um gerador para a sequência dos números primos (sem limite máximo). Por exemplo, o código
    ```python
    s = 0
    for p in primes():
        s += p
        if s > 50:
            break
    print(s)
    ```
    
    irá imprimir `58`, que é (2 + 3 + 5 + 7 + 11 + 13 + 17).